In [2]:
# Etapa 1: Query SQL para tab_gold_dim_funcionario_primeira_admissao_ultima_demissao

from pyspark.sql.functions import col, when, to_date, lit
from datetime import datetime

# ============================================================
# 0. Configurações (Modo Cirúrgico - Manual)
# ============================================================
spark.conf.set("spark.microsoft.delta.optimizeWrite.enabled", "false")
spark.conf.set("spark.microsoft.delta.autoCompact.enabled", "false")
spark.conf.set("spark.sql.adaptive.enabled", "true")

# Rebase para datas antigas
spark.conf.set("spark.sql.legacy.parquet.datetimeRebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.legacy.parquet.datetimeRebaseModeInWrite", "CORRECTED")

# Caminho físico do lakehouse de destino
path_destino = "abfss://ws_departamento_pessoal@onelake.dfs.fabric.microsoft.com/lk_departamento_pessoal.Lakehouse/Tables/tab_gold_dim_funcionario_primeira_admissao_ultima_demissao"

pfhstsit = spark.read.format("delta").load(
  "abfss://ws_dados_central@onelake.dfs.fabric.microsoft.com/lk_AutomacaoFolhaPagamento.Lakehouse/Tables/pfhstsit"
)
pfhstsit.createOrReplaceTempView("pfhstsit")

df_resultado = spark.sql("""
WITH SituacoesOrdenadas AS (
    SELECT 
        CODCOLIGADA,    
        CHAPA,
        DATAMUDANCA,
        NOVASITUACAO,
        ROW_NUMBER() OVER(PARTITION BY CODCOLIGADA, CHAPA ORDER BY DATAMUDANCA ASC) AS rn_primeira,
        ROW_NUMBER() OVER(PARTITION BY CODCOLIGADA, CHAPA ORDER BY DATAMUDANCA DESC) AS rn_ultima
    FROM 
        PFHSTSIT 
    WHERE (NOVASITUACAO = 'A' OR NOVASITUACAO = 'D')
)
SELECT 
    CODCOLIGADA,
    CHAPA,
    DATAMUDANCA,
    NOVASITUACAO
FROM SituacoesOrdenadas
WHERE (NOVASITUACAO = 'A' AND rn_primeira = 1)
   OR (NOVASITUACAO = 'D' AND rn_ultima = 1)
""")

# ================================
# Etapa 3: Correção de datas inválidas (mesma regra)
# ================================
data_minima = to_date(lit("1900-01-01"))
data_padrao = to_date(lit("2000-01-01"))

df_corrigido = (
    df_resultado
      .withColumn("DATAMUDANCA", when(col("DATAMUDANCA") < data_minima, data_padrao).otherwise(col("DATAMUDANCA")))
)

# ============================================================
# 2. ESTRATÉGIA FUNIL (Coalesce 1)
# ============================================================
df_otimizado = df_corrigido.coalesce(1)

# ============================================================
# 3. Contagem e Escrita
# ============================================================
# Realizamos a contagem para log (isso dispara uma ação leve no Spark)
qtd_linhas = df_otimizado.count()


# Escrevemos no disco
(
    df_otimizado.write
        .format("delta")
        .mode("overwrite")
        .option("overwriteSchema", "true")
        .save(path_destino)
)

agora = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"[{agora}] Sucesso! Tabela Gold atualizada.")
print(f"Total de registros: {qtd_linhas}")

StatementMeta(, e1ae160e-f8f0-4e88-a46e-91c928fd31de, 4, Finished, Available, Finished)

[2025-12-05 14:32:50] Sucesso! Tabela Gold atualizada.
Total de registros: 217559
